## Generating Context  form a Looker Explore

Use this colab to generate context  + examples from an existing Looker Explore. Make sure to connect to your Looker Instance first, Run Setup, Connect to Looker and Python Functions. Finally Run the Gradio App to use the interface.

DISCLAIMER: THIS IS SAMPLE CODE ONLY, IT IS NOT INTENDED FOR USE IN PRODUCTION ENVIRONMENTS

In [ ]:
# @title Setup
!pip install looker_sdk
#!pip install google-cloud-aiplatform --upgrade --user
!pip install pandas
!pip install -q gradio

In [ ]:
#@title Connect to Looker
from google.colab import auth
import looker_sdk
import requests
import pandas as pd
import urllib3
import os
import yaml
import string
import random
import json7
import time
import gradio as gr
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

os.environ['LOOKERSDK_BASE_URL'] = 'https://demo.looker.com'
os.environ['LOOKERSDK_CLIENT_ID'] = 'CLIENT_ID'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'CLIENT_SECRET'
os.environ['LOOKERSDK_VERIFY_SSL']= 'False'
os.environ['LOOKERSDK_API_VERSION']= '4.0'
os.environ["LOOKERSDK_TIMEOUT"] = "120"

sdk = looker_sdk.init40()

In [ ]:
#@title Python Functions : can be used independepntly.
def get_dict(model_name, explore_name):
    # API Call to pull in metadata about fields in a particular explore
    explore = sdk.lookml_model_explore(
        lookml_model_name=model_name,
        explore_name=explore_name,
        fields="id, name, description, fields, label",
    )

    measures = []
    dimensions = []

    # Iterate through the field definitions and pull in the description, sql,
    # and other looker tags you might want to include in  your data dictionary.
    if explore.fields and explore.fields.dimensions:
        for dimension in explore.fields.dimensions:
          def_dimension = {
              "name": dimension.name,
              "description": dimension.description,
          }
          dimensions.append(def_dimension)

    if explore.fields and explore.fields.measures:
        for measure in explore.fields.measures:
          def_measure = {
             "name": measure.name,
             "description": measure.description,
          }
          measures.append(def_measure)

    gr_dimensions = ""
    gr_measures = ""

    for d in dimensions:
      dim_print = d['name']
      desc_print = d['description']
      gr_dimensions = gr_dimensions +f"{dim_print}:{desc_print}\n"


    for m in measures:
      m_print = m['name']
      m_desc = m['description']
      gr_measures = gr_measures+ f"{m_print}:{m_desc}\n"

    return f"Dimensions : \n {gr_dimensions} \n Measures : \n {gr_measures}"

# Generate Standrized Examples : modify your dimension by the time dimension in your explore.
def gen_std_examples(time_dim):
  # this functions takes a standrized list of examples and replace 'events.event' by the user choosen time_dim (without the timeframe ! view.dimension eg order_items.)
  response = requests.get('https://raw.githubusercontent.com/Kriz182/explore-assistant/main/stnd_examples.txt')
  response = response.text.replace("events.event", time_dim)
  return response


## Launch Gradio App
Run the code blow to Launch the Gradio App in the Colab or Externally.
Alternatively, you can run the **functions above independently**.

In [ ]:
dwith gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # Explore Assistant : demo configuration.

    """)

    with gr.Tab(label="1. Generate Context"):
        with gr.Row():
            with gr.Column(scale=1):
                ui_model = gr.Textbox(label="Model Name")
                ui_explore = gr.Textbox(label="Explore Name")
                ui_submit_button = gr.Button(value="Generate Context")
            with gr.Column(scale=5):
                ui_context = gr.Textbox(label="Dictionary", show_copy_button=True)
                ui_submit_button.click(fn=get_dict, inputs=[ui_model, ui_explore], outputs=ui_context)

    with gr.Tab(label="2. Standrized Examples"):
        with gr.Row():
            with gr.Column(scale=1):
                ui_time_dim = gr.Textbox(label="Insert default time dimension eg `order_items.created` without the time frame", value ="view.dimension")
                ui_button_ex = ui_submit_button = gr.Button(value="Generate Examples")
            with gr.Column(scale=5):
                ui_button_ex.click(fn=gen_std_examples, inputs=ui_time_dim, outputs=gr.Textbox(label="Standrized Examples", show_copy_button=True))

    with gr.Tab(label="3. Dashboard Examples"):
        1
    with gr.Tab(label="4. Recent Queries Examples"):
        1

demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://860dcb8ee8ead54e35.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
